[FROM HERE](https://github.com/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb)

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
from ib_insync import *

util.startLoop()

ib = IB()
# Paper
ib.connect('127.0.0.1', 7496, clientId=12)

# delay data
ib.reqMarketDataType(4)

Suppose we want to find the options on the SPX, with the following conditions:

Use the next three monthly expiries;
Use strike prices within +- 20 dollar of the current SPX value;
Use strike prices that are a multitude of 5 dollar.

To avoid issues with market data permissions, we'll use delayed data:

In [3]:
ib.reqMarketDataType(4)

In [ ]:
stock_contract = Stock('TREX', 'SMART', 'USD')
stock_contract

In [ ]:
# Qualify the contract
qualified_contracts = ib.qualifyContracts(stock_contract)
stock_contract

qualified contracts - req market data - https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/

In [6]:
if not qualified_contracts:
    print("No qualified contracts found.")
else:
    stock_contract = qualified_contracts[0]

    # Request market data
    data = ib.reqMktData(stock_contract, '', False, False)
    

In [ ]:
stockValue = data.marketPrice()
stockValue

chain

In [8]:
chains = ib.reqSecDefOptParams(stock_contract.symbol, '', stock_contract.secType, stock_contract.conId)


display chains

In [ ]:
util.df(chains)

get chain SMART - tradingClass TREX

In [ ]:
chain = next(c for c in chains if c.tradingClass == 'TREX' and c.exchange == 'SMART')
chain

get strikes

In [11]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and stockValue - 30 < strike < stockValue + 30]

# print strikes

In [ ]:
strikes

get expirations from chain

In [13]:
expirations = sorted(exp for exp in chain.expirations)[:3]

# print expirations

In [ ]:
expirations

# get contract

In [ ]:
rights = ['P', 'C']

contracts = [Option('TREX', expiration, strike, right, 'SMART', tradingClass='TREX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

In [ ]:
[c for c in contracts if c.exchange == 'SMART' ]

### Get Options Chain for a Specific Symbol

In [20]:
def get_option_chain(symbol):
    underlying_stock = Stock(symbol=symbol,exchange='SMART',currency='USD')
    ib.qualifyContracts(underlying_stock)
    chains = ib.reqSecDefOptParams(underlyingSymbol=underlying_stock.symbol,futFopExchange='',underlyingSecType=underlying_stock.secType,underlyingConId=underlying_stock.conId)
    chain = [c for c in chains if c.exchange == 'SMART'][0]
    return chain

In [ ]:
# get_option_chain("TSLA")
get_option_chain("TREX")

Get Options For Specific Expiry

In [ ]:

    
def org_get_option_chain_for_expiry(symbol,expiry,min_strike=0,max_strike=np.inf):
    option_chain = get_option_chain(symbol)
    options=[]
    for strike in option_chain.strikes:
        if strike<=max_strike and strike >= min_strike:
            for right in ['C','P']:
                option = Option(symbol=symbol,lastTradeDateOrContractMonth=expiry,strike=strike,right=right,exchange='SMART',currency='USD')
                options.append(option)
    valid_options=ib.qualifyContracts(*options)
    return valid_options 

valid_options=get_option_chain_for_expiry(symbol="TREX",expiry="20250325",min_strike=70,max_strike=80)

In [34]:
def get_option_chain_for_expiry(symbol,expiry,min_strike=0,max_strike=np.inf):
    option_chain = get_option_chain(symbol)
    options=[]
    valid_options=[]
    for strike in option_chain.strikes:
        if strike<=max_strike and strike >= min_strike:
            for right in ['C','P']:
                option = Option(symbol=symbol,lastTradeDateOrContractMonth=expiry,strike=strike,right=right,exchange='SMART',currency='USD')
                valid_option=ib.qualifyContracts(option)
                valid_options.append(valid_option)
    # valid_options=ib.qualifyContracts(*options)
    return valid_options 

valid_options=get_option_chain_for_expiry(symbol="TREX",expiry="20250325",min_strike=70,max_strike=80)

Unknown contract: Option(symbol='TREX', lastTradeDateOrContractMonth='20250325', strike=70.0, right='C', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='TREX', lastTradeDateOrContractMonth='20250325', strike=70.0, right='P', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='TREX', lastTradeDateOrContractMonth='20250325', strike=72.5, right='C', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='TREX', lastTradeDateOrContractMonth='20250325', strike=72.5, right='P', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='TREX', lastTradeDateOrContractMonth='20250325', strike=75.0, right='C', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='TREX', lastTradeDateOrContractMonth='20250325', strike=75.0, right='P', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='TREX', lastTradeDateOrContractMonth='20250325', strike=77.5, right='C', exchange='SMART', currency='USD')
Unknown contract: Option(sy

In [ ]:
# valid_options=get_option_chain_for_expiry("TREX","20250325",min_strike=70,max_strike=80)
# valid_options=get_option_chain_for_expiry(symbol="TREX",expiry="20250325",min_strike=70,max_strike=80)

In [35]:
valid_options

[[], [], [], [], [], [], [], [], [], []]

Reminder

## Get maket data for these valid options for the chosen expiry date

In [ ]:
ticker_dict=[]
for option in valid_options:
    ticker_dict[(Option.right,option.strike)] = ib.reqMktData(contract=option,genericTickList="",snapshot=False,regulatorySnapshot=False)
    

### [HIER WEITER](https://www.youtube.com/watch?v=juhi5NpBBro)

## first contract

In [ ]:
contracts[0]

# Now to get the market data for all options in one go:

In [22]:
tickers = ib.reqTickers(*contracts)


In [ ]:
tickers[0]

In [ ]:
ticker = tickers[0]
ticker.midpoint




In [ ]:
ticker.ask

In [ ]:
ticker.bid

In [ ]:
ticker.marketPrice

get strike price

In [ ]:
#print(f"{ticker.contract.strike}")
ticker.contract.strike
#ticker.impliedVolatility
# deltaNeutralContract.delta

In [ ]:
type(ticker)

In [12]:
ib.disconnect()